In [1]:
!pip install pyarrow
!pip install google-cloud-bigquery

In [2]:
import psycopg2
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

### Extracting data from PostgreSQL

In [12]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'Adventureworks', user = 'postgres', password = '1234' )

In [64]:
cur = conn.cursor()
table1 = pd.read_sql_query('SELECT s.productid,p.name,	SUM(orderqty) FROM sales.salesorderdetail AS s INNER JOIN production.product AS p ON s.productid = p.productid INNER JOIN production.productcosthistory AS ph ON ph.productid = p.productid group by s.productid, p.name', conn )
table2 = pd.read_sql_query('SELECT s.stateprovinceid, sp.name, SUM(salesytd) FROM sales.salestaxrate AS s INNER JOIN person.stateprovince AS sp ON sp.stateprovinceid = s.stateprovinceid INNER JOIN sales.salesterritory AS t ON t.territoryid = sp.territoryid group by s.stateprovinceid, sp.name', conn)
table3 = pd.read_sql_query('SELECT s.specialofferid, p.name, SUM(quantity) FROM sales.specialofferproduct AS s INNER JOIN production.product AS p ON p.productid = s.productid INNER JOIN production.transactionhistory AS t ON t.productid = p.productid group by s.specialofferid, p.name', conn)
cur.close()

In [7]:
conn.close()

In [63]:
table1

,productid,name,sum
0,707,"Sport-100 Helmet, Red",18798
1,708,"Sport-100 Helmet, Black",19596
2,709,"Mountain Bike Socks, M",1107
3,710,"Mountain Bike Socks, L",90
4,711,"Sport-100 Helmet, Blue",20229
...,...,...,...
261,994,LL Bottom Bracket,378
262,996,HL Bottom Bracket,543
263,997,"Road-750 Black, 44",656
264,998,"Road-750 Black, 48",1556


In [65]:
table2

,stateprovinceid,name,sum
0,15,Florida,2.538667e+06
1,72,Tennessee,2.538667e+06
2,14,England,5.012905e+06
3,9,California,1.051085e+07
4,63,Quebec,1.354366e+07
5,41,Brunswick,6.771829e+06
6,31,Manitoba,6.771829e+06
7,69,Saskatchewan,6.771829e+06
8,57,Ontario,1.354366e+07
9,29,Labrador,6.771829e+06


In [36]:
table1.dtypes

productid     int64
name         object
sum           int64
dtype: object

In [54]:
table2.dtypes

stateprovinceid      int64
name                object
sum                float64
dtype: object

In [66]:
table3.dtypes

specialofferid     int64
name              object
sum                int64
dtype: object

In [37]:
grouping = table1

In [55]:
grouping2 = table2

In [68]:
grouping3 = table3

### Transforming data

In [43]:
grouping

,productid,name,sum
0,707,"Sport-100 Helmet, Red",18798
1,708,"Sport-100 Helmet, Black",19596
2,709,"Mountain Bike Socks, M",1107
3,710,"Mountain Bike Socks, L",90
4,711,"Sport-100 Helmet, Blue",20229
...,...,...,...
261,994,LL Bottom Bracket,378
262,996,HL Bottom Bracket,543
263,997,"Road-750 Black, 44",656
264,998,"Road-750 Black, 48",1556


In [56]:
grouping2

,stateprovinceid,name,sum
0,15,Florida,2.538667e+06
1,72,Tennessee,2.538667e+06
2,14,England,5.012905e+06
3,9,California,1.051085e+07
4,63,Quebec,1.354366e+07
5,41,Brunswick,6.771829e+06
6,31,Manitoba,6.771829e+06
7,69,Saskatchewan,6.771829e+06
8,57,Ontario,1.354366e+07
9,29,Labrador,6.771829e+06


In [69]:
grouping3

,specialofferid,name,sum
0,1,"Racing Socks, L",1949
1,1,HL Road Pedal,21413
2,1,"LL Mountain Frame - Black, 40",241
3,1,"Men's Sports Shorts, M",2
4,1,"Short-Sleeve Classic Jersey, S",1512
...,...,...,...
440,1,LL Road Pedal,45208
441,1,"Mountain-200 Black, 42",2340
442,1,"Mountain-500 Black, 42",625
443,1,"Mountain-500 Black, 44",541


### Loading data to BigQuery

In [82]:
credentials = service_account.Credentials.from_service_account_file( 
    '/Users/eduardovasquez/Downloads/taller1-315520-e65ef1c58399.json', 
    scopes = [ 'https://www.googleapis.com/auth/cloud-platform' ]
)

In [83]:
client = bigquery.Client( credentials = credentials, project = credentials.project_id )

In [84]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'productid', bigquery.enums.SqlTypeNames.INT64 ),
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'sum', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [57]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'stateprovinceid', bigquery.enums.SqlTypeNames.INT64 ),
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'sum', bigquery.enums.SqlTypeNames.FLOAT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [85]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'specialofferid', bigquery.enums.SqlTypeNames.INT64 ),
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'sum', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [86]:
job = client.load_table_from_dataframe(
    grouping, 'databases2.grouping', job_config = job_config
)
job.result()

ValueError: bq_schema contains fields not present in dataframe: {'specialofferid'}

In [87]:
job = client.load_table_from_dataframe(
    grouping2, 'databases2.grouping2', job_config = job_config
)
job.result()

ValueError: bq_schema contains fields not present in dataframe: {'specialofferid'}

In [88]:
job = client.load_table_from_dataframe(
    grouping3, 'databases2.grouping3', job_config = job_config
)
job.result()

In [81]:
table = client.get_table( 'databases2.grouping' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'databases2.grouping'
    )
)

table2 = client.get_table( 'databases2.grouping2' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table2.num_rows, len( table2.schema ), 'databases2.grouping2'
    )
)

table3 = client.get_table( 'databases2.grouping3' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table3.num_rows, len( table3.schema ), 'databases2.grouping3'
    )
)

Loaded 266 rows and 3 columns to databases2.grouping
Loaded 26 rows and 3 columns to databases2.grouping2
Loaded 445 rows and 3 columns to databases2.grouping3
